### Differentiation of tensor networks

This notebook describes examples on how to use differentiation to get the argmax of a tensornetwork. Each axis has to have only 2 dimensions!

In [2]:
import torch
import tensor

A Tensor can be created using a serialized form. 
1. tensor in serialized form:
<br>
- the value $(i_1, ... , i_k)$ in the Tensor is in place $\sum _{j=1} ^k i_j\cdot 2^{j-1}$ in the serialization
<br>
2. tensor with single axis:
<br>
- t = torch.tensor([a, b], dtype=torch.float32, requires_grad=True)
<br>
3. tensors with multiple axis:
<br>
- t = torch.tensor([serialized form of t])
- you dont need a gradient for this since only the derivatives of the single axis matter
<br>
5. tensornetwork:
<br>
- create the tensornetwork containing all tensors
<br>
5. axis:
<br>
- create a list containing lists with all axis of the tensors (the same order as in the tensornetwork)
<br>
6. contraction:
<br>
- call tensor.full_contraction_easy(., ., .)
- the first argument is a list of all tensors from which we want the derivative (these are the single axis tensors)
- the second is the tensornetwork and the third argument are the axises of the tensors

In [3]:
t_1 = torch.tensor([0, 1], dtype=torch.float32, requires_grad=True)
t_2 = torch.tensor([0, 2], dtype=torch.float32, requires_grad=True)
t_3 = torch.tensor([0, 1], dtype=torch.float32, requires_grad=True)
t_12 = torch.tensor([0, 0, 0, -3])
t_23 = torch.tensor([0, 0, 0, 2])
t_13 = torch.tensor([0, 0, 0, 1])

In [4]:
tn = [t_1, t_2, t_3, t_12, t_23, t_13]
tn_axis = [[1], [2], [3], [1, 2], [2, 3], [1, 3]]
print(tensor.full_contraction_easy([t_1, t_2, t_3], tn, tn_axis))

[tensor([1., 0.]), tensor([0., 1.]), tensor([0., 1.])]


### multiple Maxima

an example without a unique maximum. The algorithm chooses one configuration. The configuration seems to be always the same.

In [5]:
t_1 = torch.tensor([0, 1], dtype=torch.float, requires_grad=True)
t_2 = torch.tensor([0, 1], dtype=torch.float, requires_grad=True)
t_12 = torch.tensor([0, 0, 0, -1], dtype=torch.float, requires_grad=True)

In [6]:
tn = [t_1, t_2, t_12]
tn_axis = [[1], [2], [1, 2]]
print(tensor.full_contraction_easy([t_1, t_2], tn, tn_axis))

[tensor([0., 1.]), tensor([1., 0.])]


### second method for contraction

there are a lot of different possibilities for the contraction of a tensornetwork. The method above is the easiest to implement, but it should also be the slowest. You can use combination and aggregation for a single axis and this should be faster.

In [7]:
t_1 = torch.tensor([0, 1], dtype=torch.float, requires_grad=True)
t_2 = torch.tensor([0, 1], dtype=torch.float, requires_grad=True)
t_12 = torch.tensor([0, 0, 0, -1], dtype=torch.float, requires_grad=True)

tn = [t_1, t_2, t_12]
tn_axis = [[1], [2], [1, 2]]
max_axis = 2

print(tensor.full_contraction_complicated([t_1, t_2], tn, tn_axis, max_axis))

[tensor([0., 1.]), tensor([1., 0.])]


In [8]:
t_1 = torch.tensor([0, 1], dtype=torch.float32, requires_grad=True)
t_2 = torch.tensor([0, 2], dtype=torch.float32, requires_grad=True)
t_3 = torch.tensor([0, 1], dtype=torch.float32, requires_grad=True)
t_12 = torch.tensor([0, 0, 0, -3])
t_23 = torch.tensor([0, 0, 0, 2])
t_13 = torch.tensor([0, 0, 0, 1])

tn = [t_1, t_2, t_3, t_12, t_23, t_13]
tn_axis = [[1], [2], [3], [1, 2], [2, 3], [1, 3]]
max_axis = 3

print(tensor.full_contraction_complicated([t_1, t_2, t_3], tn, tn_axis, max_axis))

[tensor([1., 0.]), tensor([0., 1.]), tensor([0., 1.])]


Comparing the time of both approaches:

In [9]:
import time
import random
import itertools as it

In [10]:
def create_tensornetwork(max_axis, lower=-10, upper=10):
    # create big tensornetwork
    axis_list = [str(i + 1) for i in range(max_axis)]

    # create 2 dimensional tensors
    tn = []
    tn_axis = []
    simple_tensors = []
    for i in range(max_axis):
        t = torch.tensor([0, random.randint(lower, upper)], dtype=torch.float32, requires_grad=True)
        tn.append(t)
        simple_tensors.append(t)
        tn_axis.append([i + 1])

    # create 3 dimensional tensors
    for i in range(2, 10):
        comb = list(it.combinations(axis_list, i))
        for p in comb:
            l = [0 for _ in range(2**i)]
            l[-1] = random.randint(lower, upper)
            t = torch.tensor(l, dtype=torch.float32, requires_grad=True)
            tn.append(t)

            axis = [int(i) for i in list(p)]
            tn_axis.append(axis)

    return simple_tensors, tn, tn_axis

In [13]:
n = 20
max_axis = 12

t_1 = 0
t_2 = 0
for _ in range(n):
    simple_tensors, tn, tn_axis = create_tensornetwork(max_axis)

    start = time.time()
    tensor.full_contraction_easy(simple_tensors, tn, tn_axis)
    end = time.time()
    t_1 += end - start

    start = time.time()
    tensor.full_contraction_complicated(simple_tensors, tn, tn_axis, max_axis)
    end = time.time()
    t_2 += end - start

t_1 /= n    
t_2 /= n

print(f"time for slow contraction: {t_1}, time for fast contraction: {t_2}")

time for slow contraction: 0.6161455869674682, time for fast contraction: 0.46728992462158203
